In [1]:
from zizou.data import DataSource, GeoMagWaveforms
from zizou.ssam import SSAM 
from datetime import datetime, date, timedelta, timezone
import pandas as pd
import requests
from tonik import Storage
from io import StringIO
from obspy import UTCDateTime, Trace
import numpy as np
from zizou.visualise import plot_ssam_plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
gmc = GeoMagWaveforms(base_url='https://tilde.geonet.org.nz/v3/data',
                      method='60s', aspect='F-total-field', name='magnetic-field')
ssam = SSAM(per_lap=0.5, timestamp='start', interval=1024*60.,
            frequencies=np.linspace(1/1000., 1/120., 50))
ds = DataSource(clients=[gmc])
sg = Storage('geomag', rootdir='/tmp/geomag')
st = sg.get_substore('EYWM', '50', 'F')
startdate = UTCDateTime(2024, 4, 1)
enddate = UTCDateTime.now()
enddate = UTCDateTime(2024, 4, 9)
for tr in ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='F',
                           start=startdate, end=enddate,
                           cache=True):
    xds = ssam.compute(tr) 
    st.save(xds)

2024-10-17 22:59:43:zizou.data:3159056:755 DEBUG Data for NZ.EYWM.50.F between 2024-04-01T00:00:00.000000Z and 2024-04-09T00:00:00.000000Z not found in cache.
2024-10-17 22:59:44:zizou.data:3159056:731 DEBUG writing NZ.EYWM.50.F | 2024-04-01T00:00:00.000000Z - 2024-04-02T00:00:00.000000Z | 60.0 s, 1441 samples to /home/zizou/zizou_cache/2024/NZ/EYWM/F.D/NZ.EYWM.50.F.D.2024.092
2024-10-17 22:59:44:zizou.data:3159056:731 DEBUG writing NZ.EYWM.50.F | 2024-04-02T00:00:00.000000Z - 2024-04-03T00:00:00.000000Z | 60.0 s, 1441 samples to /home/zizou/zizou_cache/2024/NZ/EYWM/F.D/NZ.EYWM.50.F.D.2024.093
2024-10-17 22:59:44:zizou.data:3159056:731 DEBUG writing NZ.EYWM.50.F | 2024-04-03T00:00:00.000000Z - 2024-04-04T00:00:00.000000Z | 60.0 s, 1441 samples to /home/zizou/zizou_cache/2024/NZ/EYWM/F.D/NZ.EYWM.50.F.D.2024.094
2024-10-17 22:59:44:zizou.data:3159056:731 DEBUG writing NZ.EYWM.50.F | 2024-04-04T00:00:00.000000Z - 2024-04-05T00:00:00.000000Z | 60.0 s, 1441 samples to /home/zizou/zizou_cach

/workspaces/zizou/src/zizou/ssam.py:272: RuntimeWarning: divide by zero encountered in log
  hob = np.log(z / z_tot[np.newaxis, :])


In [4]:
st.starttime
spec = 

NZ.EYWM.50.F | 2024-04-01T00:00:00.000000Z - 2024-04-09T00:00:00.000000Z | 60.0 s, 11521 samples

In [ ]:
st.starttime = startdate.datetime
st.endtime = enddate.datetime
xds_st = st('filterbank')

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
tr = next(ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='F',
                           start=startdate, end=enddate))
dt = [startdate.datetime + timedelta(seconds=s) for s in tr.times()]
fig.add_trace(go.Scatter(x=dt, y=tr.data, mode='lines'), row=1, col=1)
fig.add_trace(plot_ssam_plotly(xds_st, dbscale=True, new_fig=False), row=2, col=1)